# DIAMOND + NR EC2 instance
```
Lead     : ababaian
Issue    : <github issue #>
start    : 2021 01 21
complete : YYYY MM DD
files    : NA
s3 files : s3://serratus-public/notebook/200121_nr/
```

## Introduction

As we retrieve new sequences we will have to search every sequence against the "NON REDUNDANT" protein database. Probably the fastest approach is to grab a fasta of the NR database, index it for `diamond` and throw our sequences at it

#### Links

- [BLAST in the cloud](https://blast.ncbi.nlm.nih.gov/Blast.cgi?CMD=Web&PAGE_TYPE=BlastDocs&DOC_TYPE=CloudBlast)
- [BLAST database information](https://github.com/ncbi/blast_plus_docs#blast-databases)

## EC2 Set-up script

- OS: `Amazon Linux 2 AMI (HVM) x86`
- ami: `ami-0be2609ba883822ec`
- instance: `c5.xlarge` // `r5d.4xlarge`
- description: `"c5.xlarge (- ECUs, 4 vCPUs, 3.4 GHz, -, 8 GiB memory, EBS only)"`
- description: `"r5d.4xlarge (16 vCPU	128 GB	2 x 300 NVMe SSD)"`
- storage: `450 GiB SSD (gp2)`
- encryption: `false`

In [1]:
# date and version
date

Thu Jan 21 23:50:34 PST 2021


In [ ]:
# INSTALL DIAMOND
# From base amazon linux 2
sudo yum install -y docker git

# From `serratus-align` container
mkdir diamond; cd diamond

# Install diamond2 
# Libraries for building diamond2
sudo yum -y install git gcc gcc-c++ glibc-devel \
  cmake patch automake zlib-devel make

# grab latest with fix from Benjamin
git clone https://github.com/bbuchfink/diamond.git
cd diamond

mkdir bin; cd bin
cmake ..
make -j4
sudo cp ./diamond /usr/bin/diamond
sudo chmod 755 /usr/bin/diamond

# stable copy to S3 servers
# OLD: curl https://serratus-public.s3.amazonaws.com/bin/diamond > /usr/bin/diamond2; chmod 755 /usr/bin/diamond2
# curl https://serratus-public.s3.amazonaws.com/bin/diamond > /usr/bin/diamond; chmod 755 /usr/bin/diamond


In [ ]:
# DOWNLOAD BLAST DB - NR
mkdir -p ~/nr; cd nr
wget -O ftp://ftp.ncbi.nlm.nih.gov/blast/db/FASTA/nr.gz \
 | pigz -d - \
 > nr.fa

In [ ]:
# Switch to r5d.4xlarge instance with 450 GB block storage
# Make diamond nr db
# Database hash = f0ef2411c9661667e19bf85d06ff9fab
diamond makedb -p 14 --in nr.fa -d nr

In [ ]:
# Test data (rVert unitigs)
aws s3 cp s3://serratus-public/rce/tmp/rvert_otu_analysis.tar.gz ./
tar -xvf rvert*

In [ ]:
#!/bin/bash
# run_diamond.sh
#
# Diamond search standard for Serratus blastx
# against NR database

INPUT=$1
OUTPUT=$2

# Diamond blastx alignment
time diamond blastx \
  -q  $INPUT \
  -d ~/nr/nr.dmnd \
  --masking 0 \
  --unal 1 \
  --mid-sensitive -s 1  -l 1 \
  -c1 -p14 -k1 -b 10 \
  -f 6 qseqid  qstart qend qlen qstrand \
       sseqid  sstart send slen \
       pident evalue cigar \
       qseq_translated \
  > "$OUTPUT".pro

time diamond blastx \
  -q  $INPUT \
  -d ~/nr/nr.dmnd \
  --masking 0 \
  --unal 1 \
  -l 1 \
  -c1 -p14 -k1 -b 10 \
  -f 6 qseqid  qstart qend qlen qstrand \
       sseqid  sstart send slen \
       pident evalue \
       qseq_translated \
  > "$OUTPUT".pro


# real    60m21.153s
# user    635m37.423s
# sys     5m29.373s
